In [52]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
scraper = cloudscraper.create_scraper()  # returns a requests-like object
base_url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'
base_url2 = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'
base_url3 = 'https://www.espn.com/nba/stats/team/_/season/{}/seasontype/2'

In [34]:
res1 = scraper.get(base_url.format(2025))
soup1 = BeautifulSoup(res1.text, 'html.parser')
table1 = soup1.find_all('table')[0]

    

In [35]:

headers = table1.find_all('th')[1:30]


In [36]:
res2 = scraper.get(base_url2.format(2025))
soup2 = BeautifulSoup(res2.text, 'html.parser')
table2 = soup2.find_all('table')[0]


In [89]:
res3 = scraper.get(base_url3.format(2025))
soup3 = BeautifulSoup(res3.text, 'html.parser')
table3 = soup3.find_all('table')[0]


In [88]:
headers3 = table3.find_all('th')
header_table3 = [header.text for header in headers3]


In [ ]:
header_table3.remove('RK')


In [ ]:
table4 = soup3.find_all('table', class_ = 'Table Table--align-right')[0]

<table class="Table Table--align-right" style="border-collapse:collapse;border-spacing:0"><colgroup class="Table__Colgroup"><col class="Table__Column"/><col class="Table__Column Table--sorted"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/><col class="Table__Column"/></colgroup><thead class="Table__header-group Table__THEAD"><tr class="Table__sub-header Table__TR Table__even"><th class="Table__TH" title=""><span class="" title="Games Played"><div>GP</div></span></th><th class="Table__TH" title=""><span class="underline" title="Points Per Game"><a class="AnchorLink clr-gray-01" href="/nba/sta

In [90]:
headers4 = table4.find_all('th')
header_table4 = [header.text for header in headers4]


In [87]:
combined_headers = header_table3 + header_table4
combined_headers

['Team',
 'GP',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OR',
 'DR',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TO',
 'PF']

In [37]:

headers2 = table2.find_all('th')[1:28]

In [38]:
header_table2 = [header.text for header in headers2]
header_table2.insert(1, 'Season')
header_table2.insert(2, 'id')

In [39]:
header_table = [header.text for header in headers]
header_table.insert(1, 'Season')
header_table.insert(2, 'id')


In [40]:



years = range(2015,2026)

def get_data(link, headers, slice):
    df = pd.DataFrame(columns=headers)
    for year in years:

        url = link.format(year)
        res = scraper.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        table = soup.find_all('table')[0]
        rows = table.find_all('tr')
    
        for row in rows[1:736]:
            row_data = row.find_all('td')
            individual_data = [data.text for data in row_data[slice]]
            a = row.find('a')
            player_id = ''
            
            
            if a and '/players/' in a.get('href', ''):
                href = a['href']
                player_id = href[11:-5]
            
            individual_data.insert(1, str(year)) 
            individual_data.insert(2, player_id) 
            
          
            df.loc[len(df)] = individual_data            
    return df
    

In [41]:
df = get_data(base_url, header_table, slice(0,29))


In [42]:

df2 = get_data(base_url2, header_table2, slice(0,27))
df2



,Player,Season,id,Age,Team,Pos,G,GS,MP,PER,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,James Harden,2015,hardeja01,25,HOU,SG,81,81,2981,26.7,...,14.9,31.3,12.2,4.2,16.4,.265,7.0,1.8,8.8,8.1
1,Andrew Wiggins,2015,wiggian01,19,MIN,SF,82,82,2969,13.9,...,11.7,22.6,2.1,0.0,2.1,.034,-0.4,-2.0,-2.4,-0.3
2,Trevor Ariza,2015,arizatr01,29,HOU,SF,82,82,2930,12.7,...,12.7,16.5,2.7,3.9,6.6,.108,-0.1,1.1,1.0,2.2
3,Damian Lillard,2015,lillada01,24,POR,PG,82,82,2925,20.7,...,12.6,26.9,7.3,3.3,10.6,.174,4.3,0.3,4.7,4.9
4,Chris Paul,2015,paulch01,29,LAC,PG,82,82,2857,26.0,...,12.7,23.7,12.9,3.2,16.1,.270,6.4,1.6,8.0,7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7439,Jesse Edwards,2025,edwarje01,24,MIN,C,2,0,5,-0.1,...,,0.0,0.0,0.0,0.0,.111,-8.8,4.7,-4.0,0.0
7440,James Wiseman,2025,wisemja01,23,IND,C,1,0,5,31.2,...,0.0,42.2,0.0,0.0,0.0,.237,6.0,-4.4,1.6,0.0
7441,Alondes Williams,2025,willial06,25,DET,SG,1,0,4,53.4,...,0.0,21.1,0.0,0.0,0.0,.565,37.9,6.2,44.1,0.0
7442,Zyon Pullin,2025,pullizy01,23,MEM,SG,3,0,3,-11.7,...,0.0,13.4,0.0,0.0,0.0,-0.304,-15.0,-7.7,-22.7,0.0


In [43]:
df['key'] = df['Player'] + '_' + df['Season']
df2['key'] = df2['Player'] + '_' + df2['Season']


In [44]:
df2 = df2.drop(columns=['Player', 'Season', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP','id'])

In [45]:
df2

,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,key
0,26.7,.605,.378,.561,2.8,14.2,8.5,34.6,2.6,1.6,...,31.3,12.2,4.2,16.4,.265,7.0,1.8,8.8,8.1,James Harden_2015
1,13.9,.517,.111,.410,5.0,9.5,7.2,9.8,1.5,1.3,...,22.6,2.1,0.0,2.1,.034,-0.4,-2.0,-2.4,-0.3,Andrew Wiggins_2015
2,12.7,.539,.610,.157,2.9,14.4,8.6,11.1,2.6,0.5,...,16.5,2.7,3.9,6.6,.108,-0.1,1.1,1.0,2.2,Trevor Ariza_2015
3,20.7,.560,.421,.293,1.8,11.8,6.9,29.0,1.7,0.5,...,26.9,7.3,3.3,10.6,.174,4.3,0.3,4.7,4.9,Damian Lillard_2015
4,26.0,.596,.298,.274,2.1,12.5,7.4,47.4,2.8,0.4,...,23.7,12.9,3.2,16.1,.270,6.4,1.6,8.0,7.2,Chris Paul_2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7439,-0.1,,,,0.0,0.0,0.0,23.6,0.0,0.0,...,0.0,0.0,0.0,0.0,.111,-8.8,4.7,-4.0,0.0,Jesse Edwards_2025
7440,31.2,.615,.250,.500,0.0,22.1,11.1,0.0,0.0,0.0,...,42.2,0.0,0.0,0.0,.237,6.0,-4.4,1.6,0.0,James Wiseman_2025
7441,53.4,1.250,.500,.000,0.0,0.0,0.0,65.0,0.0,0.0,...,21.1,0.0,0.0,0.0,.565,37.9,6.2,44.1,0.0,Alondes Williams_2025
7442,-11.7,.000,.000,.000,0.0,0.0,0.0,0.0,0.0,0.0,...,13.4,0.0,0.0,0.0,-0.304,-15.0,-7.7,-22.7,0.0,Zyon Pullin_2025


In [46]:
combined = pd.merge(df, df2, on='key')
combined

,Player,Season,id,Age,Team,Pos,G,GS,MP,FG,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Russell Westbrook,2015,westbru01,26,OKC,PG,67,67,34.4,9.4,...,14.3,38.4,7.5,3.2,10.6,.222,6.9,1.2,8.1,5.8
1,James Harden,2015,hardeja01,25,HOU,SG,81,81,36.8,8.0,...,14.9,31.3,12.2,4.2,16.4,.265,7.0,1.8,8.8,8.1
2,Kevin Durant,2015,duranke01,26,OKC,SF,27,27,33.8,8.8,...,12.0,29.1,3.8,1.0,4.8,.252,8.6,1.4,10.0,2.8
3,LeBron James,2015,jamesle01,30,CLE,SF,69,69,36.1,9.0,...,15.3,32.3,7.4,2.9,10.4,.199,6.1,1.0,7.1,5.7
4,Anthony Davis,2015,davisan02,21,NOP,PF,68,68,36.1,9.4,...,6.3,27.8,9.9,4.2,14.0,.274,6.6,2.3,8.9,6.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,Riley Minix,2025,minixri01,24,SAS,SF,1,0,7.0,0.0,...,0.0,6.1,0.0,0.0,0.0,-0.114,-8.5,-3.3,-11.8,0.0
12440,Jahlil Okafor,2025,okafoja01,29,IND,C,1,0,3.0,0.0,...,,0.0,0.0,0.0,0.0,.196,2.5,4.9,7.5,0.0
12441,Zyon Pullin,2025,pullizy01,23,MEM,SG,3,0,1.0,0.0,...,0.0,13.4,0.0,0.0,0.0,-0.304,-15.0,-7.7,-22.7,0.0
12442,Isaiah Stevens,2025,steveis01,24,MIA,PG,3,0,2.0,0.0,...,0.0,14.8,0.0,0.0,0.0,-0.196,-19.5,7.7,-11.8,0.0


In [47]:
combined = combined[combined['Player'] != 'League Average']


In [48]:
combined.to_csv(r'C:\Users\LL205\Documents\Machine Learning\NBA_predictions\Nba_stats.csv')